# Notebook 09 — EDA, Data Cleaning, Hypothesis Testing & Feature Engineering

**Objective:** Take the synthetic 5 000-site dataset (enriched with open-source
data where available) and prepare it for FMV modeling through four phases:

| Phase | Goal |
|-------|------|
| **2. EDA** | Understand distributions, correlations, geographic patterns, and class separability |
| **3. Data Cleaning** | Handle missing values, sentinels, outliers, and type issues |
| **4. Hypothesis Testing** | Formally test 9 hypotheses about FMV drivers with proper multiple-testing correction |
| **5. Feature Engineering** | Create 25+ derived features, screen via mutual information & VIF, and save the model-ready dataset |

**Input:** `data/synthetic/synthetic_5k_sites.parquet` (generated inline if absent)
**Outputs:**
- `data/processed/sites_clean.parquet` — cleaned dataset
- `data/processed/sites_engineered.parquet` — feature-engineered, model-ready dataset

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import (shapiro, kruskal, mannwhitneyu, spearmanr, pearsonr,
                          ttest_ind, chi2_contingency, f_oneway)
from statsmodels.stats.multitest import multipletests
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os, warnings, textwrap

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams.update({"figure.max_open_warning": 0, "figure.dpi": 110})

# Paths (relative to notebooks/ directory)
DATA_ROOT    = os.path.join("..", "data")
SYNTH_DIR    = os.path.join(DATA_ROOT, "synthetic")
PROC_DIR     = os.path.join(DATA_ROOT, "processed")
SYNTH_FILE   = os.path.join(SYNTH_DIR, "synthetic_5k_sites.parquet")
ENRICHED_CSV = os.path.join(DATA_ROOT, "census_geo_housing_combined_by_state.csv")

os.makedirs(SYNTH_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

print("Libraries loaded.")

---
## 1  Data Assembly
Load the synthetic 5 000-site dataset. If the parquet doesn't exist yet
(sim_00 not run), generate it inline using the same ground-truth FMV function.

In [ ]:
def generate_synthetic_sites(n=5000, seed=42):
    # Generate 5000 synthetic cell-tower sites with known ground-truth FMV.
    # Mirrors simulations/sim_00_data_generator.ipynb logic.
    np.random.seed(seed)
    N = n

    # ---- 0.1 Site Locations ----
    metros = [
        ('New York',40.71,-74.01,0.08,10000,75000,550000),
        ('Los Angeles',34.05,-118.24,0.07,7500,68000,650000),
        ('Chicago',41.88,-87.63,0.06,6000,62000,280000),
        ('Houston',29.76,-95.37,0.05,3500,58000,210000),
        ('Phoenix',33.45,-112.07,0.05,3000,55000,310000),
        ('Philadelphia',39.95,-75.17,0.04,5500,60000,250000),
        ('San Antonio',29.42,-98.49,0.03,3000,50000,185000),
        ('San Diego',32.72,-117.16,0.03,4000,72000,600000),
        ('Dallas',32.78,-96.80,0.05,3800,60000,250000),
        ('Atlanta',33.75,-84.39,0.04,3500,58000,260000),
        ('Miami',25.76,-80.19,0.04,6000,52000,350000),
        ('Denver',39.74,-104.99,0.03,3500,68000,450000),
        ('Seattle',47.61,-122.33,0.03,4000,80000,550000),
        ('Boston',42.36,-71.06,0.03,6500,78000,480000),
        ('Nashville',36.16,-86.78,0.03,2500,55000,280000),
        ('Portland',45.52,-122.68,0.02,3000,65000,420000),
        ('Las Vegas',36.17,-115.14,0.02,2500,52000,300000),
        ('Minneapolis',44.98,-93.27,0.02,3500,68000,280000),
        ('Charlotte',35.23,-80.84,0.02,2800,55000,250000),
        ('Austin',30.27,-97.74,0.03,3200,65000,380000),
        ('Rural Montana',46.87,-110.36,0.01,50,42000,200000),
        ('Rural Wyoming',42.75,-107.21,0.01,30,55000,225000),
        ('Rural Kansas',38.50,-98.77,0.01,40,48000,130000),
        ('Rural Alabama',32.80,-86.80,0.01,60,38000,120000),
        ('Rural Nebraska',41.50,-99.80,0.01,25,50000,140000),
        ('Rural Oregon',43.80,-120.50,0.01,35,46000,250000),
        ('Rural Maine',45.25,-69.25,0.01,40,50000,180000),
        ('Rural Midwest',40.00,-89.00,0.02,80,52000,160000),
        ('Rural South',34.00,-82.00,0.02,70,44000,150000),
        ('Rural West',37.00,-109.00,0.02,45,48000,200000),
    ]
    m_names = [m[0] for m in metros]
    m_lats  = np.array([m[1] for m in metros])
    m_lons  = np.array([m[2] for m in metros])
    m_wt    = np.array([m[3] for m in metros]); m_wt /= m_wt.sum()
    m_pop   = np.array([m[4] for m in metros])
    m_inc   = np.array([m[5] for m in metros])
    m_home  = np.array([m[6] for m in metros])

    mi = np.random.choice(len(metros), N, p=m_wt)
    spread = np.where(m_pop[mi] > 1000, 0.15, 0.6)
    lat = np.clip(m_lats[mi] + np.random.normal(0, spread), 25, 49)
    lon = np.clip(m_lons[mi] + np.random.normal(0, spread * 1.3), -125, -66)
    metro_region = [m_names[i] for i in mi]

    state_map = {
        'New York':'NY','Los Angeles':'CA','Chicago':'IL','Houston':'TX',
        'Phoenix':'AZ','Philadelphia':'PA','San Antonio':'TX','San Diego':'CA',
        'Dallas':'TX','Atlanta':'GA','Miami':'FL','Denver':'CO','Seattle':'WA',
        'Boston':'MA','Nashville':'TN','Portland':'OR','Las Vegas':'NV',
        'Minneapolis':'MN','Charlotte':'NC','Austin':'TX',
        'Rural Montana':'MT','Rural Wyoming':'WY','Rural Kansas':'KS',
        'Rural Alabama':'AL','Rural Nebraska':'NE','Rural Oregon':'OR',
        'Rural Maine':'ME','Rural Midwest':'IL','Rural South':'SC','Rural West':'NM',
    }
    state = [state_map[m_names[i]] for i in mi]

    # ---- 0.2 Property Characteristics ----
    pt_names = ['commercial','rooftop','vacant_land','residential',
                'government','agricultural','industrial']
    pt_wts   = [0.30, 0.25, 0.20, 0.12, 0.05, 0.04, 0.04]
    property_type = np.random.choice(pt_names, N, p=pt_wts)

    struct_map = {
        'commercial':['monopole','lattice','concealed_stealth'],
        'rooftop':['rooftop'],'vacant_land':['monopole','lattice','guyed'],
        'residential':['monopole','concealed_stealth'],
        'government':['monopole','lattice'],'agricultural':['lattice','guyed'],
        'industrial':['monopole','lattice'],
    }
    structure_type = [np.random.choice(struct_map[p]) for p in property_type]

    ht_p = {'monopole':(120,30),'lattice':(180,40),'guyed':(250,60),
            'rooftop':(0,0),'concealed_stealth':(80,20)}
    tower_height_ft = np.array([max(0, np.random.normal(*ht_p[s])) for s in structure_type])
    building_height_ft = np.where(
        np.isin(property_type,['rooftop','commercial']),
        np.random.lognormal(3.5,0.6,N), 0)
    lease_area_sqft = np.random.lognormal(7.5, 0.8, N)
    parcel_acreage = np.where(
        np.isin(property_type,['vacant_land','agricultural']),
        np.random.lognormal(1.5,1.0,N), lease_area_sqft/43560)
    has_building    = np.isin(property_type,['rooftop','commercial','industrial']).astype(int)
    is_ground_tower = np.isin(property_type,['vacant_land','agricultural','residential']).astype(int)
    has_parcel      = np.isin(property_type,['vacant_land','agricultural','residential']).astype(int)
    construction_year = np.random.randint(1985, 2024, N)
    construction_age  = 2026 - construction_year

    acc = {'rooftop':[0,0,.3,.7],'commercial':[.05,.15,.7,.1],'vacant_land':[.3,.4,.3,0],
           'residential':[.1,.3,.6,0],'government':[.05,.15,.7,.1],
           'agricultural':[.4,.4,.2,0],'industrial':[.1,.2,.6,.1]}
    site_access_ordinal = np.array([np.random.choice([1,2,3,4],p=acc[p]) for p in property_type])

    # ---- 0.3 Market & Demographics ----
    base_pop = m_pop[mi].astype(float)
    census_population_density = np.maximum(5, base_pop * np.random.lognormal(0,0.5,N))
    census_population_3km = (census_population_density * np.pi * 9 *
                              np.random.uniform(0.6,1.4,N)).astype(int)
    census_median_income = np.maximum(20000, m_inc[mi].astype(float) * np.random.lognormal(0,0.25,N))
    census_median_home_value = np.maximum(50000, m_home[mi].astype(float) * np.random.lognormal(0,0.3,N))
    census_median_rent = census_median_home_value * np.random.uniform(0.003,0.006,N)
    census_vacancy_rate = np.clip(np.random.beta(2,15,N), 0.01, 0.4)
    urban_rural_class = np.clip(np.digitize(census_population_density,[100,1000,5000])+1, 1, 4)
    underlying_land_value_psf = np.maximum(0.1, np.where(
        urban_rural_class>=3, np.random.lognormal(3.0,0.8,N), np.random.lognormal(1.0,0.8,N)))
    hud_safmr = census_median_rent * np.random.uniform(0.85,1.15,N)
    property_tax_rate = np.random.uniform(0.005, 0.03, N)
    topography_class = np.random.choice([1,2,3], N, p=[0.5,0.35,0.15])
    terrain_elevation_variance_m = topography_class * np.random.exponential(15, N)
    poi_density_1km = (census_population_density/50 * np.random.lognormal(0,0.5,N)).astype(int)
    building_density_500m = (census_population_density/20 * np.random.lognormal(0,0.4,N)).astype(int)
    distance_to_highway_km = np.where(urban_rural_class>=3,
        np.random.exponential(2,N), np.random.exponential(10,N))
    ground_elevation_ft = np.random.uniform(100, 8000, N)
    unemployment_rate_local = np.clip(np.random.normal(0.05,0.02,N), 0.02, 0.15)

    # ---- 0.4 Zoning, Competition & Network ----
    zon_types = ['residential','commercial','industrial','agricultural','mixed_use']
    zon_map = {'commercial':[.1,.5,.1,0,.3],'rooftop':[.2,.4,.1,0,.3],
               'vacant_land':[.2,.1,.1,.4,.2],'residential':[.6,.1,0,.1,.2],
               'government':[.1,.3,.2,.1,.3],'agricultural':[.1,0,0,.8,.1],
               'industrial':[0,.2,.6,0,.2]}
    zoning_district_type = [np.random.choice(zon_types,p=zon_map[p]) for p in property_type]
    permit_approval_difficulty = np.clip(np.random.poisson(2,N)+(urban_rural_class-1), 1, 5)
    wrluri_index = np.random.normal(0,1,N)
    wrluri_index = np.where(urban_rural_class>=3, wrluri_index+0.5, wrluri_index-0.3)
    colocation_required = (np.random.random(N)<0.25).astype(int)
    height_restriction_ft = np.where(urban_rural_class>=3,
        np.random.choice([100,120,150,200],N), np.random.choice([150,200,300,999],N))
    has_zoning_data = (np.random.random(N)<0.85).astype(int)

    fcc_tower_count_5km = np.maximum(0,
        (np.log(census_population_density+1)*3 + np.random.normal(0,2,N)).astype(int))
    fcc_tower_density_sqkm = fcc_tower_count_5km / (np.pi * 25)
    competitor_tower_distance_km = np.maximum(0.1, np.where(
        urban_rural_class>=3, np.random.exponential(1.5,N), np.random.exponential(8,N)))
    tower_per_capita = fcc_tower_count_5km / np.maximum(census_population_3km, 100) * 1000
    scarcity_index = np.clip(
        1 - (1/(fcc_tower_count_5km+1))*(topography_class/3)*(permit_approval_difficulty/5), 0, 1)
    landlord_concentration = np.maximum(1, (fcc_tower_count_5km*0.6+np.random.poisson(2,N)).astype(int))
    composite_hazard_score = np.random.beta(2, 8, N)

    tenant_count = np.clip(np.random.poisson(1.5,N)+1, 1, 5)
    primary_technology = np.random.choice([1,2,3,4,5], N, p=[0.02,0.05,0.35,0.45,0.13])
    coverage_critical = (np.random.random(N) < np.where(urban_rural_class<=2, 0.30, 0.08)).astype(int)
    capacity_utilization_pct = np.clip(np.random.beta(3,4,N)*100, 5, 99)
    high_traffic = (capacity_utilization_pct > 70).astype(int)
    backhaul_type = np.random.choice([1,2,3,4], N, p=[0.03,0.07,0.25,0.65])
    has_fiber_connection = (backhaul_type==4).astype(int)
    network_substitutability_score = np.clip(np.random.beta(4,3,N)*(1-coverage_critical*0.5), 0, 1)
    search_ring_alternative_count = np.maximum(0,
        (fcc_tower_count_5km*0.3+np.random.poisson(1,N)-coverage_critical*2).astype(int))
    inter_site_distance_km = np.maximum(0.2, np.where(
        urban_rural_class>=3, np.random.exponential(1.2,N), np.random.exponential(5,N)))
    antenna_count = tenant_count * np.random.choice([3,4,6], N, p=[0.3,0.5,0.2])
    structure_capacity_remaining = np.clip(np.random.beta(3,2,N)-tenant_count*0.15, 0, 1)
    site_priority_tier = np.where(coverage_critical==1, 1,
                         np.where(capacity_utilization_pct>60, 2, 3))

    # ---- 0.5 TRUE FMV ----
    log_fmv = (4.5
        + 0.35*np.log(census_median_home_value)
        + 0.20*np.log(census_population_density+1)
        + 0.15*np.log(underlying_land_value_psf+1)
        + 0.10*tenant_count
        + 0.25*coverage_critical
        - 0.10*np.log(fcc_tower_count_5km+1)
        + 0.15*scarcity_index
        + 0.08*(permit_approval_difficulty-3)
        + 0.05*has_fiber_connection
        + np.random.normal(0, 0.08, N))
    true_fmv = np.exp(log_fmv)
    scale = 1500 / np.median(true_fmv)
    true_fmv = np.clip(true_fmv * scale, 200, 8000)

    # ---- 0.6 Distorted Rents ----
    lease_age_years = np.random.uniform(2, 25, N)
    total_term = np.random.choice([5,10,15,20,25], N, p=[.1,.2,.3,.25,.15])
    remaining_term_years = np.maximum(0, total_term - lease_age_years + np.random.uniform(0,5,N))
    esc_types = ['fixed_3pct','fixed_2pct','cpi_linked','flat']
    escalation_type = np.random.choice(esc_types, N, p=[.40,.20,.25,.15])
    esc_rate = {'fixed_3pct':0.03,'fixed_2pct':0.02,'cpi_linked':0.025,'flat':0.0}
    escalation_rate = np.array([esc_rate[e] for e in escalation_type])
    origination_factor = np.random.lognormal(0, 0.10, N)
    fmv_inception = true_fmv / (1.02 ** lease_age_years)
    base_rent = fmv_inception * origination_factor
    cumulative_escalation_factor = (1 + escalation_rate) ** lease_age_years
    current_monthly_rent = base_rent * cumulative_escalation_factor
    amend = np.random.random(N) < 0.05
    amendment_count = amend.astype(int)
    current_monthly_rent[amend] = true_fmv[amend] * np.random.lognormal(0, 0.05, amend.sum())
    de_escalated_base_rent = current_monthly_rent / cumulative_escalation_factor
    current_monthly_rent = np.clip(current_monthly_rent, 100, 15000)
    true_deviation_pct = (current_monthly_rent - true_fmv) / true_fmv
    true_class = np.where(true_deviation_pct > 0.15, 'OVERPAYING',
                 np.where(true_deviation_pct < -0.15, 'UNDERPAYING', 'AT_MARKET'))

    # ---- 0.7 Assemble ----
    df = pd.DataFrame({
        'site_id': [f'SIM_{i:05d}' for i in range(N)],
        'lat': lat, 'lon': lon, 'state': state, 'metro_region': metro_region,
        'property_type': property_type, 'structure_type': structure_type,
        'tower_height_ft': np.round(tower_height_ft,1),
        'building_height_ft': np.round(building_height_ft,1),
        'lease_area_sqft': np.round(lease_area_sqft,0),
        'parcel_acreage': np.round(parcel_acreage,2),
        'construction_year': construction_year, 'construction_age': construction_age,
        'site_access_ordinal': site_access_ordinal,
        'has_building': has_building, 'is_ground_tower': is_ground_tower, 'has_parcel': has_parcel,
        'zoning_district_type': zoning_district_type,
        'permit_approval_difficulty': permit_approval_difficulty,
        'wrluri_index': np.round(wrluri_index,3),
        'colocation_required': colocation_required,
        'height_restriction_ft': height_restriction_ft,
        'has_zoning_data': has_zoning_data,
        'census_population_density': np.round(census_population_density,1),
        'census_population_3km': census_population_3km,
        'census_median_income': np.round(census_median_income,0),
        'census_median_home_value': np.round(census_median_home_value,0),
        'census_median_rent': np.round(census_median_rent,0),
        'census_vacancy_rate': np.round(census_vacancy_rate,4),
        'urban_rural_class': urban_rural_class,
        'underlying_land_value_psf': np.round(underlying_land_value_psf,2),
        'hud_safmr': np.round(hud_safmr,0),
        'property_tax_rate': np.round(property_tax_rate,4),
        'topography_class': topography_class,
        'terrain_elevation_variance_m': np.round(terrain_elevation_variance_m,1),
        'poi_density_1km': poi_density_1km,
        'building_density_500m': building_density_500m,
        'distance_to_highway_km': np.round(distance_to_highway_km,2),
        'ground_elevation_ft': np.round(ground_elevation_ft,1),
        'unemployment_rate_local': np.round(unemployment_rate_local,4),
        'fcc_tower_count_5km': fcc_tower_count_5km,
        'fcc_tower_density_sqkm': np.round(fcc_tower_density_sqkm,4),
        'competitor_tower_distance_km': np.round(competitor_tower_distance_km,2),
        'tower_per_capita': np.round(tower_per_capita,4),
        'scarcity_index': np.round(scarcity_index,4),
        'landlord_concentration': landlord_concentration,
        'composite_hazard_score': np.round(composite_hazard_score,4),
        'tenant_count': tenant_count, 'primary_technology': primary_technology,
        'coverage_critical': coverage_critical,
        'capacity_utilization_pct': np.round(capacity_utilization_pct,1),
        'high_traffic': high_traffic, 'backhaul_type': backhaul_type,
        'has_fiber_connection': has_fiber_connection,
        'network_substitutability_score': np.round(network_substitutability_score,4),
        'search_ring_alternative_count': search_ring_alternative_count,
        'inter_site_distance_km': np.round(inter_site_distance_km,2),
        'antenna_count': antenna_count,
        'structure_capacity_remaining': np.round(structure_capacity_remaining,4),
        'site_priority_tier': site_priority_tier,
        'lease_age_years': np.round(lease_age_years,2),
        'remaining_term_years': np.round(remaining_term_years,2),
        'escalation_type': escalation_type,
        'escalation_rate': escalation_rate,
        'cumulative_escalation_factor': np.round(cumulative_escalation_factor,4),
        'amendment_count': amendment_count,
        'de_escalated_base_rent': np.round(de_escalated_base_rent,2),
        'current_monthly_rent': np.round(current_monthly_rent,2),
        'true_fmv': np.round(true_fmv,2),
        'true_deviation_pct': np.round(true_deviation_pct,4),
        'true_class': true_class,
    })
    return df

# ---------- Load or generate ----------
if os.path.exists(SYNTH_FILE):
    df = pd.read_parquet(SYNTH_FILE)
    print(f"Loaded existing synthetic data: {df.shape}")
else:
    print("Synthetic file not found — generating 5 000 sites inline...")
    df = generate_synthetic_sites()
    os.makedirs(os.path.dirname(SYNTH_FILE), exist_ok=True)
    df.to_parquet(SYNTH_FILE, index=False)
    print(f"Generated and saved: {df.shape}")

print(f"\nColumns ({len(df.columns)}): {list(df.columns)[:10]} ... {list(df.columns)[-5:]}")

In [ ]:
# Try to load the combined enrichment data (state-level) from notebooks 06-08
enrichment_loaded = False
if os.path.exists(ENRICHED_CSV):
    df_enrich = pd.read_csv(ENRICHED_CSV)
    print(f"Enrichment data loaded: {df_enrich.shape}")
    print(f"  Columns: {list(df_enrich.columns)}")

    # Merge enrichment data onto synthetic sites by state abbreviation
    # The enrichment data has 'state_abbr'; synthetic has 'state'
    if 'state_abbr' in df_enrich.columns:
        # Use latest year for static merge
        if 'year' in df_enrich.columns:
            latest_yr = df_enrich['year'].max()
            df_enrich_latest = df_enrich[df_enrich['year'] == latest_yr].copy()
            print(f"  Using latest year: {latest_yr}")
        else:
            df_enrich_latest = df_enrich.copy()

        cols_to_merge = [c for c in df_enrich_latest.columns
                         if c not in ('year', 'state_name', 'state_fips')]
        df_enrich_latest = df_enrich_latest[cols_to_merge].drop_duplicates(subset='state_abbr')

        n_before = len(df)
        df = df.merge(df_enrich_latest, left_on='state', right_on='state_abbr',
                       how='left', suffixes=('', '_enr'))
        if 'state_abbr' in df.columns and 'state' in df.columns:
            df.drop(columns=['state_abbr'], inplace=True, errors='ignore')
        enrichment_loaded = True
        print(f"  Merged: {n_before} -> {len(df)} rows, {len(df.columns)} cols")
    else:
        print("  WARNING: 'state_abbr' not found in enrichment data — skipping merge.")
else:
    print("No enrichment data found (run notebooks 06-08 first). Proceeding with synthetic features only.")
    print(f"  Looked for: {os.path.abspath(ENRICHED_CSV)}")

In [ ]:
print("=" * 70)
print("DATASET OVERVIEW")
print("=" * 70)
print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"\nColumn types:")
print(df.dtypes.value_counts().to_string())

# Separate column groups
ID_COLS   = ['site_id', 'lat', 'lon', 'state', 'metro_region']
CAT_COLS  = ['property_type', 'structure_type', 'zoning_district_type',
             'escalation_type', 'true_class']
BIN_COLS  = ['has_building', 'is_ground_tower', 'has_parcel', 'colocation_required',
             'has_zoning_data', 'coverage_critical', 'high_traffic', 'has_fiber_connection']
ORD_COLS  = ['site_access_ordinal', 'urban_rural_class', 'topography_class',
             'primary_technology', 'backhaul_type', 'site_priority_tier']
TARGET    = 'current_monthly_rent'
GROUND_TRUTH = ['true_fmv', 'true_deviation_pct', 'true_class']
NUM_COLS  = [c for c in df.columns if c not in ID_COLS + CAT_COLS + BIN_COLS + ORD_COLS + GROUND_TRUTH + [TARGET]]

print(f"\nFeature groups:")
print(f"  Identity:      {len(ID_COLS)}")
print(f"  Categorical:   {len(CAT_COLS)}")
print(f"  Binary:        {len(BIN_COLS)}")
print(f"  Ordinal:       {len(ORD_COLS)}")
print(f"  Numeric:       {len(NUM_COLS)}")
print(f"  Target:        1 ({TARGET})")
print(f"  Ground truth:  {len(GROUND_TRUTH)}")

df.describe(percentiles=[.01, .05, .25, .50, .75, .95, .99]).T.round(2)

In [ ]:
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
miss_df = pd.DataFrame({'missing_count': missing, 'missing_pct': missing_pct})
miss_df = miss_df[miss_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)

if len(miss_df) == 0:
    print("No missing values in any column.")
else:
    print(f"Columns with missing values ({len(miss_df)}):")
    print(miss_df.to_string())

# Also check for potential sentinel values (-666666666 from Census, -999, etc.)
sentinel_check = {}
for col in df.select_dtypes(include=[np.number]).columns:
    neg_count = (df[col] < 0).sum()
    if neg_count > 0 and col not in ('lon', 'true_deviation_pct', 'wrluri_index'):
        sentinel_check[col] = neg_count
if sentinel_check:
    print(f"\nColumns with negative values (potential sentinels):")
    for col, cnt in sorted(sentinel_check.items(), key=lambda x: -x[1]):
        print(f"  {col}: {cnt} negative values (min={df[col].min():.2f})")
else:
    print("\nNo suspicious sentinel values detected.")

---
## 2  Exploratory Data Analysis

Deep-dive into distributions, correlations, geographic patterns, and the
relationship between features and the target variable (`current_monthly_rent`)
and ground-truth FMV (`true_fmv`).

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Current rent distribution
axes[0,0].hist(df['current_monthly_rent'], bins=60, color='steelblue',
               edgecolor='white', alpha=0.8)
axes[0,0].axvline(df['current_monthly_rent'].median(), color='red', ls='--', label='Median')
axes[0,0].set_title('Current Monthly Rent')
axes[0,0].set_xlabel('$/month')
axes[0,0].legend()

# 2. Log rent
log_rent = np.log(df['current_monthly_rent'])
axes[0,1].hist(log_rent, bins=60, color='coral', edgecolor='white', alpha=0.8)
axes[0,1].set_title('log(Current Monthly Rent)')
axes[0,1].set_xlabel('log($/month)')

# 3. True FMV vs Current Rent
colors_map = {'OVERPAYING': 'red', 'AT_MARKET': 'green', 'UNDERPAYING': 'blue'}
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    mask = df['true_class'] == cls
    axes[0,2].scatter(df.loc[mask, 'true_fmv'], df.loc[mask, 'current_monthly_rent'],
                      alpha=0.25, s=5, c=colors_map[cls], label=cls)
axes[0,2].plot([0, 8000], [0, 8000], 'k--', lw=1)
axes[0,2].set_xlabel('True FMV ($/mo)')
axes[0,2].set_ylabel('Current Rent ($/mo)')
axes[0,2].set_title('Rent vs True FMV')
axes[0,2].legend(fontsize=8, markerscale=3)

# 4. Deviation distribution
axes[1,0].hist(df['true_deviation_pct'] * 100, bins=60, color='mediumpurple',
               edgecolor='white', alpha=0.8)
axes[1,0].axvline(0, color='black', ls='--')
axes[1,0].set_title('Rent Deviation from FMV (%)')
axes[1,0].set_xlabel('Deviation %')

# 5. Class distribution
class_counts = df['true_class'].value_counts()
axes[1,1].bar(class_counts.index, class_counts.values,
              color=['red','green','blue'], alpha=0.7)
axes[1,1].set_title('FMV Class Distribution')
axes[1,1].set_ylabel('Count')

# 6. QQ-plot for log(rent)
stats.probplot(log_rent, plot=axes[1,2])
axes[1,2].set_title('Q-Q Plot: log(Rent)')

plt.suptitle('Target Variable Analysis', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

# Normality test on log(rent)
stat, p = shapiro(log_rent.sample(2000, random_state=42))
print(f"Shapiro-Wilk on log(rent): W={stat:.4f}, p={p:.6f}")
print(f"  -> {'Approximately normal' if p > 0.05 else 'Not normal'} at alpha=0.05")
print(f"\nRent summary: mean=${df['current_monthly_rent'].mean():,.0f}, "
      f"median=${df['current_monthly_rent'].median():,.0f}, "
      f"std=${df['current_monthly_rent'].std():,.0f}")
print(f"Skewness: {df['current_monthly_rent'].skew():.3f}, "
      f"Kurtosis: {df['current_monthly_rent'].kurtosis():.3f}")

In [ ]:
key_continuous = [
    'census_median_income', 'census_median_home_value', 'census_population_density',
    'underlying_land_value_psf', 'hud_safmr', 'lease_area_sqft',
    'tower_height_ft', 'distance_to_highway_km', 'fcc_tower_count_5km',
    'capacity_utilization_pct', 'wrluri_index', 'lease_age_years',
    'terrain_elevation_variance_m', 'unemployment_rate_local',
    'inter_site_distance_km', 'competitor_tower_distance_km',
]

fig, axes = plt.subplots(4, 4, figsize=(20, 16))
axes = axes.ravel()

for i, col in enumerate(key_continuous):
    ax = axes[i]
    data = df[col].dropna()
    # Use log scale for highly skewed features
    skew = data.skew()
    if abs(skew) > 2 and data.min() > 0:
        ax.hist(np.log1p(data), bins=50, color='steelblue', edgecolor='white', alpha=0.7)
        ax.set_xlabel(f'log({col})')
    else:
        ax.hist(data, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
        ax.set_xlabel(col)
    ax.set_title(f'{col}\nsk={skew:.2f}', fontsize=9)
    ax.tick_params(labelsize=7)

plt.suptitle('Univariate Distributions — Key Continuous Features', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Property type
df['property_type'].value_counts().plot.bar(ax=axes[0,0], color='teal', alpha=0.7)
axes[0,0].set_title('Property Type')
axes[0,0].tick_params(axis='x', rotation=45)

# Structure type
df['structure_type'].value_counts().plot.bar(ax=axes[0,1], color='coral', alpha=0.7)
axes[0,1].set_title('Structure Type')
axes[0,1].tick_params(axis='x', rotation=45)

# Escalation type
df['escalation_type'].value_counts().plot.bar(ax=axes[0,2], color='mediumpurple', alpha=0.7)
axes[0,2].set_title('Escalation Type')
axes[0,2].tick_params(axis='x', rotation=45)

# Urban/Rural class
df['urban_rural_class'].value_counts().sort_index().plot.bar(ax=axes[1,0], color='olive', alpha=0.7)
axes[1,0].set_title('Urban/Rural Class')
axes[1,0].set_xlabel('1=Rural, 2=Small Town, 3=Suburban, 4=Urban Core')

# Topography class
df['topography_class'].value_counts().sort_index().plot.bar(ax=axes[1,1], color='sienna', alpha=0.7)
axes[1,1].set_title('Topography Class')
axes[1,1].set_xlabel('1=Flat, 2=Hilly, 3=Mountainous')

# Zoning district type
df['zoning_district_type'].value_counts().plot.bar(ax=axes[1,2], color='steelblue', alpha=0.7)
axes[1,2].set_title('Zoning District Type')
axes[1,2].tick_params(axis='x', rotation=45)

plt.suptitle('Categorical Feature Distributions', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Select top features by absolute correlation with rent
num_features = df.select_dtypes(include=[np.number]).columns.tolist()
exclude = ['true_fmv', 'true_deviation_pct']  # don't leak ground truth
num_for_corr = [c for c in num_features if c not in exclude]

corr_with_rent = df[num_for_corr].corrwith(df['current_monthly_rent']).abs().sort_values(ascending=False)
top_20 = corr_with_rent.head(20).index.tolist()
if 'current_monthly_rent' not in top_20:
    top_20 = ['current_monthly_rent'] + top_20[:19]

corr_matrix = df[top_20].corr()

fig, ax = plt.subplots(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5, ax=ax,
            annot_kws={'size': 8})
ax.set_title('Pearson Correlation — Top 20 Features vs Rent', fontsize=13)
plt.tight_layout()
plt.show()

print("Top 10 features correlated with current_monthly_rent:")
for feat, corr_val in corr_with_rent.head(10).items():
    direction = '+' if df[feat].corr(df['current_monthly_rent']) > 0 else '-'
    print(f"  {direction} {feat}: {corr_val:.3f}")

In [ ]:
top_predictors = ['census_median_home_value', 'census_population_density',
                  'underlying_land_value_psf', 'tenant_count',
                  'hud_safmr', 'lease_age_years']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, col in enumerate(top_predictors):
    ax = axes[i]
    x = df[col]
    y = df['current_monthly_rent']
    # Use log scale for skewed features
    if x.skew() > 2 and x.min() > 0:
        x_plot = np.log1p(x)
        xlabel = f'log({col})'
    else:
        x_plot = x
        xlabel = col

    ax.scatter(x_plot, y, alpha=0.15, s=5, c='steelblue')
    # Add lowess trend line
    from scipy.ndimage import uniform_filter1d
    sort_idx = np.argsort(x_plot.values)
    x_sorted = x_plot.values[sort_idx]
    y_sorted = y.values[sort_idx]
    y_smooth = uniform_filter1d(y_sorted, size=200)
    ax.plot(x_sorted, y_smooth, 'r-', lw=2, label='Smoothed trend')
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Rent ($/mo)')
    # Spearman correlation
    rho, p_val = spearmanr(x, y)
    ax.set_title(f'{col}\nSpearman rho={rho:.3f}, p={p_val:.2e}', fontsize=10)
    ax.legend(fontsize=8)

plt.suptitle('Key Predictors vs Current Monthly Rent', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Median rent by state
state_stats = df.groupby('state').agg(
    median_rent=('current_monthly_rent', 'median'),
    median_fmv=('true_fmv', 'median'),
    count=('site_id', 'count'),
    median_deviation=('true_deviation_pct', 'median')
).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Bar chart — rent by state
state_stats_sorted = state_stats.sort_values('median_rent', ascending=False)
axes[0].barh(state_stats_sorted['state'], state_stats_sorted['median_rent'],
             color='steelblue', alpha=0.7)
axes[0].set_xlabel('Median Monthly Rent ($)')
axes[0].set_title('Median Rent by State')
axes[0].invert_yaxis()

# Scatter map
sc = axes[1].scatter(df['lon'], df['lat'], c=df['current_monthly_rent'],
                     cmap='YlOrRd', s=3, alpha=0.4, vmin=500, vmax=4000)
plt.colorbar(sc, ax=axes[1], label='Rent ($/mo)')
axes[1].set_title('Geographic Distribution (colored by rent)')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')

plt.tight_layout()
plt.show()

# Rent by urban/rural class
print("\nRent by Urban/Rural Class:")
urban_stats = df.groupby('urban_rural_class')['current_monthly_rent'].describe()[['count','mean','50%','std']]
urban_stats.index = ['Rural', 'Small Town', 'Suburban', 'Urban Core']
print(urban_stats.round(0).to_string())

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# 1. Rent by property type — violin plot
order = df.groupby('property_type')['current_monthly_rent'].median().sort_values(ascending=False).index
sns.violinplot(data=df, x='property_type', y='current_monthly_rent', order=order,
               palette='Set2', ax=axes[0], cut=0)
axes[0].set_title('Rent Distribution by Property Type')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_ylim(0, 6000)

# 2. FMV deviation by property type
sns.boxplot(data=df, x='property_type', y='true_deviation_pct', order=order,
            palette='Set2', ax=axes[1], showfliers=False)
axes[1].axhline(0, color='black', ls='--', lw=1)
axes[1].set_title('FMV Deviation (%) by Property Type')
axes[1].set_xlabel('')
axes[1].tick_params(axis='x', rotation=45)

# 3. Class proportion by property type
ct = pd.crosstab(df['property_type'], df['true_class'], normalize='index') * 100
ct = ct.reindex(order)
ct[['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']].plot.bar(
    stacked=True, ax=axes[2], color=['red','green','blue'], alpha=0.7)
axes[2].set_title('FMV Class % by Property Type')
axes[2].set_ylabel('Percentage')
axes[2].set_xlabel('')
axes[2].tick_params(axis='x', rotation=45)
axes[2].legend(fontsize=8)

plt.tight_layout()
plt.show()

# Kruskal-Wallis test
groups = [g['current_monthly_rent'].values for _, g in df.groupby('property_type')]
kw_stat, kw_p = kruskal(*groups)
print(f"Kruskal-Wallis across property types: H={kw_stat:.2f}, p={kw_p:.2e}")

# Eta-squared (effect size for ANOVA)
grand_mean = df['current_monthly_rent'].mean()
ss_between = sum(len(g) * (g['current_monthly_rent'].mean() - grand_mean)**2
                 for _, g in df.groupby('property_type'))
ss_total = ((df['current_monthly_rent'] - grand_mean)**2).sum()
eta_sq = ss_between / ss_total
print(f"Eta-squared (property type -> rent): {eta_sq:.4f} ({eta_sq*100:.2f}% of variance)")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Tower count vs rent
axes[0,0].scatter(df['fcc_tower_count_5km'], df['current_monthly_rent'],
                  alpha=0.1, s=3, c='steelblue')
axes[0,0].set_xlabel('FCC Tower Count (5km)')
axes[0,0].set_ylabel('Rent ($/mo)')
rho, _ = spearmanr(df['fcc_tower_count_5km'], df['current_monthly_rent'])
axes[0,0].set_title(f'Tower Count vs Rent (rho={rho:.3f})')

# 2. Competitor distance vs rent
axes[0,1].scatter(np.log1p(df['competitor_tower_distance_km']),
                  df['current_monthly_rent'], alpha=0.1, s=3, c='coral')
axes[0,1].set_xlabel('log(Competitor Distance km)')
axes[0,1].set_ylabel('Rent ($/mo)')
rho, _ = spearmanr(df['competitor_tower_distance_km'], df['current_monthly_rent'])
axes[0,1].set_title(f'Competitor Distance vs Rent (rho={rho:.3f})')

# 3. Coverage critical boxplot
sns.boxplot(data=df, x='coverage_critical', y='current_monthly_rent',
            palette=['lightblue','salmon'], ax=axes[0,2])
axes[0,2].set_title('Coverage Critical vs Rent')
axes[0,2].set_xticklabels(['Not Critical', 'Critical'])

# 4. Tenant count vs rent
sns.boxplot(data=df, x='tenant_count', y='current_monthly_rent',
            palette='viridis', ax=axes[1,0])
axes[1,0].set_title('Tenant Count vs Rent')

# 5. Technology generation vs rent
tech_labels = {1:'2G', 2:'3G', 3:'4G', 4:'5G sub6', 5:'5G mmW'}
df_tech = df.copy()
df_tech['tech_label'] = df_tech['primary_technology'].map(tech_labels)
sns.boxplot(data=df_tech, x='tech_label', y='current_monthly_rent',
            order=['2G','3G','4G','5G sub6','5G mmW'], palette='coolwarm', ax=axes[1,1])
axes[1,1].set_title('Technology vs Rent')

# 6. Fiber connection vs rent
sns.boxplot(data=df, x='has_fiber_connection', y='current_monthly_rent',
            palette=['lightblue','salmon'], ax=axes[1,2])
axes[1,2].set_title('Fiber Connection vs Rent')
axes[1,2].set_xticklabels(['No Fiber', 'Fiber'])

plt.suptitle('Competition & Network Features vs Rent', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Lease age distribution
axes[0,0].hist(df['lease_age_years'], bins=50, color='mediumpurple', edgecolor='white', alpha=0.7)
axes[0,0].set_title('Lease Age Distribution')
axes[0,0].set_xlabel('Years')

# 2. Lease age vs deviation
axes[0,1].scatter(df['lease_age_years'], df['true_deviation_pct']*100,
                  alpha=0.1, s=3, c='mediumpurple')
axes[0,1].axhline(0, color='black', ls='--', lw=1)
axes[0,1].set_xlabel('Lease Age (years)')
axes[0,1].set_ylabel('Deviation from FMV (%)')
rho, p_val = spearmanr(df['lease_age_years'], df['true_deviation_pct'].abs())
axes[0,1].set_title(f'Lease Age vs |Deviation| (rho={rho:.3f})')

# 3. Escalation type vs deviation
sns.boxplot(data=df, x='escalation_type', y='true_deviation_pct',
            palette='Set2', ax=axes[0,2], showfliers=False)
axes[0,2].axhline(0, color='black', ls='--')
axes[0,2].set_title('Escalation Type vs Deviation')
axes[0,2].tick_params(axis='x', rotation=30)

# 4. Cumulative escalation factor
axes[1,0].hist(df['cumulative_escalation_factor'], bins=50, color='teal',
               edgecolor='white', alpha=0.7)
axes[1,0].set_title('Cumulative Escalation Factor')

# 5. De-escalated vs current rent
axes[1,1].scatter(df['de_escalated_base_rent'], df['current_monthly_rent'],
                  alpha=0.1, s=3, c='teal')
axes[1,1].plot([0,8000],[0,8000], 'k--', lw=1)
axes[1,1].set_xlabel('De-escalated Base Rent')
axes[1,1].set_ylabel('Current Rent')
axes[1,1].set_title('De-escalated vs Current Rent')

# 6. Amendment impact
sns.boxplot(data=df, x='amendment_count', y='true_deviation_pct',
            palette=['lightblue','salmon'], ax=axes[1,2], showfliers=False)
axes[1,2].axhline(0, color='black', ls='--')
axes[1,2].set_title('Amendment vs Deviation')
axes[1,2].set_xticklabels(['No Amendment', 'Amended'])

plt.suptitle('Lease & Contract Feature Analysis', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
key_feats = ['census_median_home_value', 'census_population_density',
             'tenant_count', 'fcc_tower_count_5km', 'capacity_utilization_pct',
             'lease_age_years']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feat in enumerate(key_feats):
    ax = axes[i]
    for cls, color in [('OVERPAYING','red'),('AT_MARKET','green'),('UNDERPAYING','blue')]:
        data = df.loc[df['true_class']==cls, feat].dropna()
        ax.hist(data, bins=40, alpha=0.4, color=color, label=cls, density=True)
    ax.set_title(feat, fontsize=10)
    ax.legend(fontsize=7)

plt.suptitle('Feature Distributions by FMV Class', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

# Class separation summary
print("Feature means by class (normalized):")
class_means = df.groupby('true_class')[key_feats].mean()
class_means_norm = (class_means - class_means.mean()) / class_means.std()
print(class_means_norm.round(3).to_string())

In [ ]:
print("=" * 70)
print("EDA KEY FINDINGS SUMMARY")
print("=" * 70)

findings = [
    f"1. Rent is right-skewed (skew={df['current_monthly_rent'].skew():.2f}); "
    f"log(rent) is approximately normal.",
    f"2. Median rent: ${df['current_monthly_rent'].median():,.0f}/mo, "
    f"IQR: ${df['current_monthly_rent'].quantile(0.25):,.0f}-"
    f"${df['current_monthly_rent'].quantile(0.75):,.0f}.",
    f"3. Class split: "
    + ", ".join(f"{c}={n} ({n/len(df)*100:.1f}%)"
                for c, n in df['true_class'].value_counts().items()),
    f"4. Top correlates with rent: census_median_home_value, "
    f"census_median_income, underlying_land_value_psf.",
    f"5. Property type explains {eta_sq*100:.1f}% of rent variance (eta-squared).",
    f"6. Urban cores have {df.loc[df['urban_rural_class']==4, 'current_monthly_rent'].median()/df.loc[df['urban_rural_class']==1, 'current_monthly_rent'].median():.1f}x "
    f"higher median rent than rural areas.",
    f"7. Coverage-critical sites command a rent premium.",
    f"8. Older leases show greater absolute deviation from FMV (escalation drift).",
    f"9. Competition (tower count) is positively correlated with rent "
    f"(collinear with population density).",
]
for f in findings:
    print(f"\n{f}")

---
## 3  Data Cleaning

Systematic treatment of missing values, sentinel values, outliers, and
type inconsistencies. The goal is a clean dataset suitable for clustering
and supervised modeling.

In [ ]:
# Work on a copy
dfc = df.copy()
n_before = len(dfc)
print(f"Starting shape: {dfc.shape}")

# ---- 3a. Missing values ----
total_missing = dfc.isnull().sum().sum()
print(f"\nTotal missing values: {total_missing}")

if total_missing > 0:
    # Strategy: median impute numeric, mode impute categorical
    for col in dfc.columns:
        if dfc[col].isnull().sum() > 0:
            n_miss = dfc[col].isnull().sum()
            if dfc[col].dtype in ['float64', 'int64']:
                fill_val = dfc[col].median()
                dfc[col].fillna(fill_val, inplace=True)
                print(f"  {col}: {n_miss} missing -> median imputed ({fill_val:.2f})")
            else:
                fill_val = dfc[col].mode()[0]
                dfc[col].fillna(fill_val, inplace=True)
                print(f"  {col}: {n_miss} missing -> mode imputed ({fill_val})")
else:
    print("  No missing values to impute.")

# Verify
assert dfc.isnull().sum().sum() == 0, "Missing values remain!"
print("\nAll missing values handled.")

In [ ]:
print("=" * 70)
print("SENTINEL & IMPOSSIBLE VALUE DETECTION")
print("=" * 70)

# Define valid ranges for key features
valid_ranges = {
    'census_population_density': (0, 100000),
    'census_median_income':      (5000, 500000),
    'census_median_home_value':  (10000, 5000000),
    'census_median_rent':        (50, 10000),
    'census_vacancy_rate':       (0, 1),
    'unemployment_rate_local':   (0, 0.5),
    'tower_height_ft':           (0, 2000),
    'building_height_ft':        (0, 2000),
    'lease_area_sqft':           (10, 500000),
    'parcel_acreage':            (0, 10000),
    'current_monthly_rent':      (50, 20000),
    'capacity_utilization_pct':  (0, 100),
    'hud_safmr':                 (50, 10000),
    'property_tax_rate':         (0, 0.10),
    'ground_elevation_ft':       (0, 15000),
    'distance_to_highway_km':    (0, 200),
    'lease_age_years':           (0, 50),
    'remaining_term_years':      (0, 50),
}

violations = {}
for col, (lo, hi) in valid_ranges.items():
    if col in dfc.columns:
        n_lo = (dfc[col] < lo).sum()
        n_hi = (dfc[col] > hi).sum()
        if n_lo + n_hi > 0:
            violations[col] = {'below': n_lo, 'above': n_hi}
            # Clip to valid range
            dfc[col] = dfc[col].clip(lo, hi)

if violations:
    print("Range violations found and clipped:")
    for col, v in violations.items():
        print(f"  {col}: {v['below']} below min, {v['above']} above max")
else:
    print("No range violations found.")

# Check for Census sentinel -666666666
for col in dfc.select_dtypes(include=[np.number]).columns:
    n_sentinel = ((dfc[col] == -666666666) | (dfc[col] == -999)).sum()
    if n_sentinel > 0:
        print(f"\n  SENTINEL in {col}: {n_sentinel} values -> replaced with median")
        dfc.loc[(dfc[col] == -666666666) | (dfc[col] == -999), col] = dfc[col].median()

print("\nSentinel/impossible value check complete.")

In [ ]:
# Outlier detection using IQR method on key numeric features
outlier_cols = ['current_monthly_rent', 'census_median_income',
                'census_median_home_value', 'underlying_land_value_psf',
                'lease_area_sqft', 'tower_height_ft', 'hud_safmr']

print("=" * 70)
print("OUTLIER DETECTION (IQR Method, 3x IQR)")
print("=" * 70)

outlier_summary = {}
for col in outlier_cols:
    Q1 = dfc[col].quantile(0.25)
    Q3 = dfc[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 3 * IQR
    upper = Q3 + 3 * IQR
    n_outliers = ((dfc[col] < lower) | (dfc[col] > upper)).sum()
    outlier_summary[col] = {
        'Q1': Q1, 'Q3': Q3, 'IQR': IQR,
        'lower': lower, 'upper': upper,
        'n_outliers': n_outliers,
        'pct': n_outliers / len(dfc) * 100
    }
    print(f"  {col}: {n_outliers} outliers ({n_outliers/len(dfc)*100:.2f}%) "
          f"[{lower:.0f}, {upper:.0f}]")

# Strategy: Winsorize (cap at 1st/99th percentile) rather than remove
print("\nWinsorizing at 1st and 99th percentiles...")
for col in outlier_cols:
    p01 = dfc[col].quantile(0.01)
    p99 = dfc[col].quantile(0.99)
    n_clipped = ((dfc[col] < p01) | (dfc[col] > p99)).sum()
    dfc[col] = dfc[col].clip(p01, p99)
    if n_clipped > 0:
        print(f"  {col}: {n_clipped} values winsorized to [{p01:.0f}, {p99:.0f}]")

# Visualize before/after for rent
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].hist(df['current_monthly_rent'], bins=60, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].set_title('Rent — Before Cleaning')
axes[0].set_xlabel('$/month')
axes[1].hist(dfc['current_monthly_rent'], bins=60, color='coral', edgecolor='white', alpha=0.7)
axes[1].set_title('Rent — After Winsorization')
axes[1].set_xlabel('$/month')
plt.tight_layout()
plt.show()

In [ ]:
# Ensure correct dtypes
int_cols = ['construction_year', 'construction_age', 'site_access_ordinal',
            'has_building', 'is_ground_tower', 'has_parcel', 'colocation_required',
            'has_zoning_data', 'urban_rural_class', 'topography_class',
            'poi_density_1km', 'building_density_500m', 'fcc_tower_count_5km',
            'landlord_concentration', 'tenant_count', 'primary_technology',
            'coverage_critical', 'high_traffic', 'backhaul_type',
            'has_fiber_connection', 'search_ring_alternative_count',
            'antenna_count', 'site_priority_tier', 'amendment_count',
            'height_restriction_ft', 'census_population_3km']

for col in int_cols:
    if col in dfc.columns:
        dfc[col] = dfc[col].astype(int)

# Verify no duplicates
n_dup = dfc.duplicated(subset='site_id').sum()
print(f"Duplicate site_ids: {n_dup}")
if n_dup > 0:
    dfc = dfc.drop_duplicates(subset='site_id')
    print(f"  Removed {n_dup} duplicates")

# Summary
print(f"\nClean dataset: {dfc.shape}")
print(f"  Numeric cols: {len(dfc.select_dtypes(include=[np.number]).columns)}")
print(f"  Object cols:  {len(dfc.select_dtypes(include=['object']).columns)}")
print(f"  Null values:  {dfc.isnull().sum().sum()}")

In [ ]:
# Save cleaned dataset
clean_path = os.path.join(PROC_DIR, "sites_clean.parquet")
dfc.to_parquet(clean_path, index=False)
print(f"Saved cleaned dataset: {clean_path}")
print(f"  Shape: {dfc.shape}")
print(f"  Size: {os.path.getsize(clean_path) / 1024:.0f} KB")

---
## 4  Experiment Design & Hypothesis Testing

We formulate and test 9 hypotheses about what drives cell tower lease FMV.
Each test includes the test statistic, p-value, effect size, and practical
interpretation. We apply Benjamini-Hochberg FDR correction at the end.

### Hypotheses

| # | Hypothesis | Test | Effect Size |
|---|-----------|------|-------------|
| H1 | Rent differs across property types | Kruskal-Wallis | Eta-squared |
| H2 | Urban sites command higher rent | Mann-Whitney U | Cohen's d |
| H3 | Older leases deviate more from FMV | Spearman rho | rho |
| H4 | Coverage-critical sites have higher FMV | Mann-Whitney U | Cohen's d |
| H5 | More tower competition lowers FMV | Spearman partial rho | rho |
| H6 | Demographics (income, home value) drive FMV | Multiple regression R² | Partial R |
| H7 | Fiber connectivity commands a premium | Mann-Whitney U | Cohen's d |
| H8 | Higher regulatory burden (WRLURI) increases rent | Spearman rho | rho |
| H9 | Higher hazard score lowers FMV | Spearman rho | rho |

In [ ]:
# H1: Rent distributions differ significantly across property types
print("=" * 70)
print("H1: Rent differs significantly across property types")
print("=" * 70)

groups_h1 = [g['current_monthly_rent'].values for _, g in dfc.groupby('property_type')]
kw_stat, kw_p = kruskal(*groups_h1)

# Effect size: eta-squared
grand_mean = dfc['current_monthly_rent'].mean()
ss_between = sum(len(g) * (g['current_monthly_rent'].mean() - grand_mean)**2
                 for _, g in dfc.groupby('property_type'))
ss_total = ((dfc['current_monthly_rent'] - grand_mean)**2).sum()
eta_sq_h1 = ss_between / ss_total

print(f"  Test: Kruskal-Wallis H")
print(f"  H-statistic: {kw_stat:.2f}")
print(f"  p-value: {kw_p:.2e}")
print(f"  Eta-squared: {eta_sq_h1:.4f} ({eta_sq_h1*100:.2f}% of variance)")
print(f"  -> {'REJECT H0' if kw_p < 0.05 else 'FAIL TO REJECT H0'}: "
      f"Rent {'differs' if kw_p < 0.05 else 'does not differ'} across property types.")

# Pairwise comparisons (Bonferroni)
prop_types = dfc['property_type'].unique()
n_pairs = len(prop_types) * (len(prop_types) - 1) // 2
print(f"\n  Pairwise Mann-Whitney U ({n_pairs} pairs, Bonferroni alpha={0.05/n_pairs:.4f}):")
sig_pairs = 0
for i in range(len(prop_types)):
    for j in range(i+1, len(prop_types)):
        g1 = dfc.loc[dfc['property_type']==prop_types[i], 'current_monthly_rent']
        g2 = dfc.loc[dfc['property_type']==prop_types[j], 'current_monthly_rent']
        u_stat, u_p = mannwhitneyu(g1, g2, alternative='two-sided')
        if u_p < 0.05 / n_pairs:
            sig_pairs += 1
print(f"  {sig_pairs}/{n_pairs} pairs significantly different after Bonferroni correction.")

h1_result = {'hypothesis': 'H1', 'test': 'Kruskal-Wallis',
             'statistic': kw_stat, 'p_value': kw_p,
             'effect_size': eta_sq_h1, 'effect_metric': 'eta-squared',
             'decision': 'Reject' if kw_p < 0.05 else 'Fail to reject'}

In [ ]:
# H2: Urban sites (class 3-4) command significantly higher rent than rural (class 1-2)
print("=" * 70)
print("H2: Urban sites command higher rent than rural")
print("=" * 70)

urban_rent = dfc.loc[dfc['urban_rural_class'] >= 3, 'current_monthly_rent']
rural_rent = dfc.loc[dfc['urban_rural_class'] <= 2, 'current_monthly_rent']

u_stat, u_p = mannwhitneyu(urban_rent, rural_rent, alternative='greater')

# Cohen's d
pooled_std = np.sqrt((urban_rent.std()**2 + rural_rent.std()**2) / 2)
cohens_d = (urban_rent.mean() - rural_rent.mean()) / pooled_std

print(f"  Urban (n={len(urban_rent)}): mean=${urban_rent.mean():,.0f}, median=${urban_rent.median():,.0f}")
print(f"  Rural (n={len(rural_rent)}): mean=${rural_rent.mean():,.0f}, median=${rural_rent.median():,.0f}")
print(f"  Test: Mann-Whitney U (one-sided: urban > rural)")
print(f"  U-statistic: {u_stat:,.0f}")
print(f"  p-value: {u_p:.2e}")
print(f"  Cohen's d: {cohens_d:.3f} ({'small' if abs(cohens_d)<0.5 else 'medium' if abs(cohens_d)<0.8 else 'large'})")
print(f"  -> {'REJECT H0' if u_p < 0.05 else 'FAIL TO REJECT H0'}")

h2_result = {'hypothesis': 'H2', 'test': 'Mann-Whitney U',
             'statistic': u_stat, 'p_value': u_p,
             'effect_size': cohens_d, 'effect_metric': "Cohen's d",
             'decision': 'Reject' if u_p < 0.05 else 'Fail to reject'}

In [ ]:
# H3: Older leases deviate more from true FMV (escalation drift)
print("=" * 70)
print("H3: Older leases have greater absolute FMV deviation")
print("=" * 70)

rho_h3, p_h3 = spearmanr(dfc['lease_age_years'], dfc['true_deviation_pct'].abs())

# Additional: correlate lease age with signed deviation
rho_signed, p_signed = spearmanr(dfc['lease_age_years'], dfc['true_deviation_pct'])

print(f"  Spearman rho (lease age vs |deviation|): {rho_h3:.4f}, p={p_h3:.2e}")
print(f"  Spearman rho (lease age vs signed deviation): {rho_signed:.4f}, p={p_signed:.2e}")
print(f"  -> {'REJECT H0' if p_h3 < 0.05 else 'FAIL TO REJECT H0'}: "
      f"Older leases {'do' if p_h3 < 0.05 else 'do not'} deviate more.")

# Binned analysis
bins = [0, 5, 10, 15, 20, 25]
labels = ['0-5', '5-10', '10-15', '15-20', '20-25']
dfc['_age_bin'] = pd.cut(dfc['lease_age_years'], bins=bins, labels=labels)
age_stats = dfc.groupby('_age_bin', observed=True).agg(
    mean_abs_dev=('true_deviation_pct', lambda x: x.abs().mean()),
    median_abs_dev=('true_deviation_pct', lambda x: x.abs().median()),
    overpaying_pct=('true_class', lambda x: (x == 'OVERPAYING').mean() * 100)
).round(4)
print(f"\n  Deviation by lease age bin:")
print(age_stats.to_string())
dfc.drop(columns=['_age_bin'], inplace=True)

h3_result = {'hypothesis': 'H3', 'test': 'Spearman',
             'statistic': rho_h3, 'p_value': p_h3,
             'effect_size': rho_h3, 'effect_metric': 'Spearman rho',
             'decision': 'Reject' if p_h3 < 0.05 else 'Fail to reject'}

In [ ]:
# H4: Coverage-critical sites have significantly higher TRUE FMV
print("=" * 70)
print("H4: Coverage-critical sites have higher FMV")
print("=" * 70)

fmv_critical = dfc.loc[dfc['coverage_critical'] == 1, 'true_fmv']
fmv_noncrit  = dfc.loc[dfc['coverage_critical'] == 0, 'true_fmv']

u_stat, u_p = mannwhitneyu(fmv_critical, fmv_noncrit, alternative='greater')
pooled_std = np.sqrt((fmv_critical.std()**2 + fmv_noncrit.std()**2) / 2)
cohens_d = (fmv_critical.mean() - fmv_noncrit.mean()) / pooled_std

print(f"  Critical (n={len(fmv_critical)}): mean=${fmv_critical.mean():,.0f}, median=${fmv_critical.median():,.0f}")
print(f"  Non-critical (n={len(fmv_noncrit)}): mean=${fmv_noncrit.mean():,.0f}, median=${fmv_noncrit.median():,.0f}")
print(f"  Premium: ${fmv_critical.mean() - fmv_noncrit.mean():,.0f}/mo ({(fmv_critical.mean()/fmv_noncrit.mean()-1)*100:.1f}%)")
print(f"  Mann-Whitney U: {u_stat:,.0f}, p={u_p:.2e}")
print(f"  Cohen's d: {cohens_d:.3f}")
print(f"  -> {'REJECT H0' if u_p < 0.05 else 'FAIL TO REJECT H0'}")

h4_result = {'hypothesis': 'H4', 'test': 'Mann-Whitney U',
             'statistic': u_stat, 'p_value': u_p,
             'effect_size': cohens_d, 'effect_metric': "Cohen's d",
             'decision': 'Reject' if u_p < 0.05 else 'Fail to reject'}

In [ ]:
# H5: More competition (tower count) lowers FMV, controlling for population density
print("=" * 70)
print("H5: Tower competition lowers FMV (controlling for population density)")
print("=" * 70)

# Simple Spearman
rho_simple, p_simple = spearmanr(dfc['fcc_tower_count_5km'], dfc['true_fmv'])
print(f"  Simple Spearman (tower_count vs FMV): rho={rho_simple:.4f}, p={p_simple:.2e}")

# Partial correlation: tower_count vs FMV, controlling for population_density
# Method: correlate residuals after regressing each on the control
from numpy.polynomial.polynomial import polyfit
x = dfc['fcc_tower_count_5km'].values.astype(float)
y = dfc['true_fmv'].values.astype(float)
z = dfc['census_population_density'].values.astype(float)

# Residualize x on z
coef_xz = np.polyfit(z, x, 1)
res_x = x - np.polyval(coef_xz, z)

# Residualize y on z
coef_yz = np.polyfit(z, y, 1)
res_y = y - np.polyval(coef_yz, z)

rho_partial, p_partial = spearmanr(res_x, res_y)
print(f"  Partial Spearman (tower_count vs FMV | pop_density): rho={rho_partial:.4f}, p={p_partial:.2e}")
print(f"\n  Interpretation: After controlling for population density,")
print(f"  tower competition has a {'negative' if rho_partial < 0 else 'positive'} "
      f"association with FMV (rho={rho_partial:.4f}).")
print(f"  -> {'REJECT H0' if p_partial < 0.05 else 'FAIL TO REJECT H0'}")

h5_result = {'hypothesis': 'H5', 'test': 'Partial Spearman',
             'statistic': rho_partial, 'p_value': p_partial,
             'effect_size': rho_partial, 'effect_metric': 'Partial rho',
             'decision': 'Reject' if p_partial < 0.05 else 'Fail to reject'}

In [ ]:
# H6: Demographic features (income, home value, land value) jointly explain FMV
print("=" * 70)
print("H6: Demographics significantly predict FMV")
print("=" * 70)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

demo_features = ['census_median_income', 'census_median_home_value',
                 'underlying_land_value_psf', 'census_population_density']

X_demo = dfc[demo_features].values
y_fmv  = dfc['true_fmv'].values

# Full model
lr_full = LinearRegression().fit(X_demo, y_fmv)
r2_full = r2_score(y_fmv, lr_full.predict(X_demo))

# Individual partial R-squared (drop-one approach)
print(f"  Full model R-squared: {r2_full:.4f}")
print(f"\n  Partial R-squared (drop-one):")
for i, feat in enumerate(demo_features):
    X_reduced = np.delete(X_demo, i, axis=1)
    lr_reduced = LinearRegression().fit(X_reduced, y_fmv)
    r2_reduced = r2_score(y_fmv, lr_reduced.predict(X_reduced))
    partial_r2 = r2_full - r2_reduced
    print(f"    {feat}: partial R² = {partial_r2:.4f}")

# F-test for joint significance
from scipy.stats import f as f_dist
n = len(y_fmv)
k = len(demo_features)
f_stat = (r2_full / k) / ((1 - r2_full) / (n - k - 1))
f_p = 1 - f_dist.cdf(f_stat, k, n - k - 1)
print(f"\n  F-test (joint significance): F={f_stat:.2f}, p={f_p:.2e}")
print(f"  -> {'REJECT H0' if f_p < 0.05 else 'FAIL TO REJECT H0'}: "
      f"Demographics {'jointly predict' if f_p < 0.05 else 'do not predict'} FMV.")

h6_result = {'hypothesis': 'H6', 'test': 'F-test (OLS)',
             'statistic': f_stat, 'p_value': f_p,
             'effect_size': r2_full, 'effect_metric': 'R-squared',
             'decision': 'Reject' if f_p < 0.05 else 'Fail to reject'}

In [ ]:
# H7: Sites with fiber connectivity have higher FMV
print("=" * 70)
print("H7: Fiber connectivity commands an FMV premium")
print("=" * 70)

fmv_fiber    = dfc.loc[dfc['has_fiber_connection'] == 1, 'true_fmv']
fmv_no_fiber = dfc.loc[dfc['has_fiber_connection'] == 0, 'true_fmv']

u_stat, u_p = mannwhitneyu(fmv_fiber, fmv_no_fiber, alternative='greater')
pooled_std = np.sqrt((fmv_fiber.std()**2 + fmv_no_fiber.std()**2) / 2)
cohens_d = (fmv_fiber.mean() - fmv_no_fiber.mean()) / pooled_std

print(f"  Fiber (n={len(fmv_fiber)}): mean=${fmv_fiber.mean():,.0f}, median=${fmv_fiber.median():,.0f}")
print(f"  No fiber (n={len(fmv_no_fiber)}): mean=${fmv_no_fiber.mean():,.0f}, median=${fmv_no_fiber.median():,.0f}")
print(f"  Premium: ${fmv_fiber.mean()-fmv_no_fiber.mean():,.0f}/mo ({(fmv_fiber.mean()/fmv_no_fiber.mean()-1)*100:.1f}%)")
print(f"  Mann-Whitney U: {u_stat:,.0f}, p={u_p:.2e}")
print(f"  Cohen's d: {cohens_d:.3f} ({'small' if abs(cohens_d)<0.5 else 'medium' if abs(cohens_d)<0.8 else 'large'})")
print(f"  -> {'REJECT H0' if u_p < 0.05 else 'FAIL TO REJECT H0'}")

h7_result = {'hypothesis': 'H7', 'test': 'Mann-Whitney U',
             'statistic': u_stat, 'p_value': u_p,
             'effect_size': cohens_d, 'effect_metric': "Cohen's d",
             'decision': 'Reject' if u_p < 0.05 else 'Fail to reject'}

In [ ]:
# H8: Higher WRLURI (regulatory restrictiveness) is associated with higher rent
print("=" * 70)
print("H8: Regulatory burden (WRLURI) increases rent")
print("=" * 70)

rho_h8, p_h8 = spearmanr(dfc['wrluri_index'], dfc['current_monthly_rent'])
rho_fmv, p_fmv = spearmanr(dfc['wrluri_index'], dfc['true_fmv'])

print(f"  Spearman (WRLURI vs rent): rho={rho_h8:.4f}, p={p_h8:.2e}")
print(f"  Spearman (WRLURI vs FMV):  rho={rho_fmv:.4f}, p={p_fmv:.2e}")

# Control for urban/rural (since WRLURI correlates with urbanity)
# Partial correlation
z_urban = dfc['urban_rural_class'].values.astype(float)
x_wrluri = dfc['wrluri_index'].values.astype(float)
y_rent = dfc['current_monthly_rent'].values.astype(float)

coef_xz = np.polyfit(z_urban, x_wrluri, 1)
res_x = x_wrluri - np.polyval(coef_xz, z_urban)
coef_yz = np.polyfit(z_urban, y_rent, 1)
res_y = y_rent - np.polyval(coef_yz, z_urban)
rho_partial, p_partial = spearmanr(res_x, res_y)

print(f"  Partial Spearman (WRLURI vs rent | urban_rural): rho={rho_partial:.4f}, p={p_partial:.2e}")
print(f"  -> {'REJECT H0' if p_partial < 0.05 else 'FAIL TO REJECT H0'}")

h8_result = {'hypothesis': 'H8', 'test': 'Partial Spearman',
             'statistic': rho_partial, 'p_value': p_partial,
             'effect_size': rho_partial, 'effect_metric': 'Partial rho',
             'decision': 'Reject' if p_partial < 0.05 else 'Fail to reject'}

In [ ]:
# H9: Higher composite hazard score is associated with lower FMV
print("=" * 70)
print("H9: Higher hazard score lowers FMV")
print("=" * 70)

rho_h9, p_h9 = spearmanr(dfc['composite_hazard_score'], dfc['true_fmv'])

# Split into high-hazard (top quartile) vs low-hazard (bottom quartile)
q75_hazard = dfc['composite_hazard_score'].quantile(0.75)
q25_hazard = dfc['composite_hazard_score'].quantile(0.25)
fmv_high_haz = dfc.loc[dfc['composite_hazard_score'] >= q75_hazard, 'true_fmv']
fmv_low_haz  = dfc.loc[dfc['composite_hazard_score'] <= q25_hazard, 'true_fmv']

u_stat, u_p = mannwhitneyu(fmv_low_haz, fmv_high_haz, alternative='greater')
pooled_std = np.sqrt((fmv_high_haz.std()**2 + fmv_low_haz.std()**2) / 2)
cohens_d = (fmv_low_haz.mean() - fmv_high_haz.mean()) / pooled_std

print(f"  Spearman (hazard vs FMV): rho={rho_h9:.4f}, p={p_h9:.2e}")
print(f"  High-hazard FMV (n={len(fmv_high_haz)}): mean=${fmv_high_haz.mean():,.0f}")
print(f"  Low-hazard FMV (n={len(fmv_low_haz)}): mean=${fmv_low_haz.mean():,.0f}")
print(f"  Mann-Whitney U (low > high): U={u_stat:,.0f}, p={u_p:.2e}")
print(f"  Cohen's d: {cohens_d:.3f}")
print(f"  -> {'REJECT H0' if p_h9 < 0.05 else 'FAIL TO REJECT H0'}")

h9_result = {'hypothesis': 'H9', 'test': 'Spearman + Mann-Whitney',
             'statistic': rho_h9, 'p_value': p_h9,
             'effect_size': rho_h9, 'effect_metric': 'Spearman rho',
             'decision': 'Reject' if p_h9 < 0.05 else 'Fail to reject'}

In [ ]:
# Collect all hypothesis results
all_results = [h1_result, h2_result, h3_result, h4_result,
               h5_result, h6_result, h7_result, h8_result, h9_result]

df_hyp = pd.DataFrame(all_results)

# Benjamini-Hochberg FDR correction
reject_bh, p_corrected, _, _ = multipletests(df_hyp['p_value'], method='fdr_bh', alpha=0.05)
df_hyp['p_corrected_bh'] = p_corrected
df_hyp['reject_bh'] = reject_bh

# Bonferroni correction
reject_bonf, p_bonf, _, _ = multipletests(df_hyp['p_value'], method='bonferroni', alpha=0.05)
df_hyp['p_corrected_bonf'] = p_bonf
df_hyp['reject_bonf'] = reject_bonf

print("=" * 90)
print("HYPOTHESIS TESTING SUMMARY (with multiple testing correction)")
print("=" * 90)
display_cols = ['hypothesis', 'test', 'statistic', 'p_value', 'effect_size',
                'effect_metric', 'decision', 'reject_bh', 'reject_bonf']
print(df_hyp[display_cols].to_string(index=False,
    formatters={
        'statistic': lambda x: f'{x:.3f}',
        'p_value': lambda x: f'{x:.2e}',
        'effect_size': lambda x: f'{x:.4f}',
        'p_corrected_bh': lambda x: f'{x:.2e}',
    }))

n_reject = df_hyp['reject_bh'].sum()
print(f"\n{n_reject}/9 hypotheses rejected after BH-FDR correction (alpha=0.05)")
print(f"{df_hyp['reject_bonf'].sum()}/9 hypotheses rejected after Bonferroni correction")

In [ ]:
print("=" * 70)
print("EXPERIMENT DESIGN: A/B Framework for Lease Renegotiation")
print("=" * 70)

lines_obj = [
    "OBJECTIVE",
    "---------",
    "Design a controlled experiment to validate the FMV model renegotiation",
    "recommendations before portfolio-wide deployment.",
    "",
    "DESIGN: Stratified Randomized Controlled Trial",
    "-----------------------------------------------",
    "1. POPULATION: All leases classified as OVERPAYING by the model.",
    "",
    "2. STRATIFICATION:",
    "   - Property type (7 strata)",
    "   - Urban/rural class (4 strata)",
    "   - Lease age tercile (3 strata)",
    "   -> Up to 84 stratification cells",
    "",
    "3. RANDOMIZATION:",
    "   - Within each stratum, randomly assign leases to:",
    "     - Treatment (A): Renegotiate using FMV-based target rent",
    "     - Control (B): Continue existing lease terms",
    "   - 50/50 split within each stratum",
    "",
    "4. PRIMARY OUTCOME:",
    "   - Delta_rent = rent_after - rent_before ($/month)",
    "   - Measured 12 months post-renegotiation",
    "",
    "5. SECONDARY OUTCOMES:",
    "   - Renegotiation success rate (binary)",
    "   - Time to close renegotiation (days)",
    "   - Landlord satisfaction score (ordinal)",
    "",
    "6. SAMPLE SIZE CALCULATION:",
]
print("\n".join(lines_obj))

# Power analysis for the A/B test
from scipy.stats import norm
alpha_power = 0.05
power = 0.80
effect_size_target = 200  # $200/month reduction target
sigma = dfc.loc[dfc['true_class']=='OVERPAYING', 'current_monthly_rent'].std()

z_alpha = norm.ppf(1 - alpha_power / 2)
z_beta = norm.ppf(power)
n_per_group = int(np.ceil(2 * ((z_alpha + z_beta) * sigma / effect_size_target) ** 2))

print(f"   Target effect: ${effect_size_target}/month reduction")
print(f"   Assumed sigma: ${sigma:,.0f}")
print(f"   Alpha: {alpha_power}, Power: {power}")
print(f"   Required N per group: {n_per_group}")
print(f"   Total N needed: {2 * n_per_group}")
n_overpaying = (dfc['true_class'] == 'OVERPAYING').sum()
print(f"   Available OVERPAYING leases: {n_overpaying}")
print(f"   Feasibility: {'FEASIBLE' if n_overpaying >= 2*n_per_group else 'NEED MORE SITES'}")

lines_plan = [
    "7. ANALYSIS PLAN:",
    "   - Primary: Two-sample t-test on Delta_rent (Treatment vs Control)",
    "   - Robustness: Wilcoxon rank-sum (non-parametric)",
    "   - Heterogeneity: Subgroup analysis by property type and urban/rural",
    "   - Multiple testing: Bonferroni correction for subgroup analyses",
    "",
    "8. DECISION RULE:",
    "   - If p < 0.05 and mean savings > $100/month: DEPLOY portfolio-wide",
    "   - If p < 0.05 but savings < $100/month: Re-evaluate cost-benefit",
    "   - If p >= 0.05: Do NOT deploy; investigate model calibration",
]
print("\n".join(lines_plan))

---
## 5  Feature Engineering

Transform and combine raw features into model-ready predictors. The goal is
to capture non-linear relationships, interactions, and domain-specific
knowledge that improve FMV prediction accuracy.

**Strategy:**
1. Log transforms for highly skewed features
2. Interaction features capturing domain knowledge
3. Ratio features normalising by local market context
4. Composite indices combining related signals
5. Categorical encoding (target-free)
6. Feature screening via mutual information and VIF

In [ ]:
# Start from clean dataset
dfe = dfc.copy()
print(f"Starting feature engineering: {dfe.shape}")

# ---- 5.1 Log Transforms for Skewed Features ----
log_candidates = [
    'census_population_density', 'census_median_income',
    'census_median_home_value', 'underlying_land_value_psf',
    'lease_area_sqft', 'hud_safmr', 'census_median_rent',
    'distance_to_highway_km', 'competitor_tower_distance_km',
    'inter_site_distance_km', 'terrain_elevation_variance_m',
    'parcel_acreage',
]

log_created = []
for col in log_candidates:
    if col in dfe.columns:
        skew_before = dfe[col].skew()
        new_col = f'log_{col}'
        dfe[new_col] = np.log1p(dfe[col])
        skew_after = dfe[new_col].skew()
        log_created.append(new_col)

print(f"Log transforms created: {len(log_created)}")
for col in log_created:
    orig = col.replace('log_', '')
    print(f"  {col}: skew {dfe[orig].skew():.2f} -> {dfe[col].skew():.2f}")

In [ ]:
# ---- 5.2 Interaction Features ----
# Domain-motivated interactions from FMV coefficient analysis

# Coverage criticality amplifies scarcity premium
dfe['coverage_x_scarcity'] = dfe['coverage_critical'] * dfe['scarcity_index']

# Fiber amplifies coverage value
dfe['fiber_x_coverage'] = dfe['has_fiber_connection'] * dfe['coverage_critical']

# Market value context for coverage
dfe['coverage_x_home_value'] = dfe['coverage_critical'] * np.log1p(dfe['census_median_home_value'])

# Urban density amplifies competition effect
dfe['urban_x_tower_count'] = dfe['urban_rural_class'] * dfe['fcc_tower_count_5km']

# Permit difficulty in high-demand areas
dfe['permit_x_pop_density'] = dfe['permit_approval_difficulty'] * np.log1p(dfe['census_population_density'])

# Multi-tenant value in urban settings
dfe['tenant_x_urban'] = dfe['tenant_count'] * (dfe['urban_rural_class'] >= 3).astype(int)

# Lease age interacted with escalation rate (drift proxy)
dfe['age_x_escalation'] = dfe['lease_age_years'] * dfe['escalation_rate']

# Height interacted with building
dfe['height_premium'] = dfe['tower_height_ft'] + dfe['building_height_ft']

interaction_cols = ['coverage_x_scarcity', 'fiber_x_coverage', 'coverage_x_home_value',
                    'urban_x_tower_count', 'permit_x_pop_density', 'tenant_x_urban',
                    'age_x_escalation', 'height_premium']
print(f"Interaction features created: {len(interaction_cols)}")
for col in interaction_cols:
    print(f"  {col}: mean={dfe[col].mean():.3f}, std={dfe[col].std():.3f}")

In [ ]:
# ---- 5.3 Ratio Features ----
# These normalize absolute values by local context

# Rent-to-income ratio (affordability)
dfe['rent_to_income_ratio'] = dfe['current_monthly_rent'] / (dfe['census_median_income'] / 12 + 1)

# Rent-to-home-value ratio (yield)
dfe['rent_to_home_value_ratio'] = dfe['current_monthly_rent'] / (dfe['census_median_home_value'] / 12 + 1)

# Rent-to-land-value ratio
dfe['rent_to_land_value'] = dfe['current_monthly_rent'] / (dfe['underlying_land_value_psf'] * dfe['lease_area_sqft'] / 12 + 1)

# Tower density per capita (competition intensity)
dfe['tower_density_per_capita'] = dfe['fcc_tower_count_5km'] / (dfe['census_population_3km'] / 1000 + 0.1)

# SAFMR premium (how much above market rent)
dfe['safmr_premium'] = dfe['current_monthly_rent'] / (dfe['hud_safmr'] + 1)

# Capacity per tenant (network load per carrier)
dfe['capacity_per_tenant'] = dfe['capacity_utilization_pct'] / (dfe['tenant_count'] + 0.1)

# Height-to-restriction ratio (how close to limit)
dfe['height_utilization'] = dfe['tower_height_ft'] / (dfe['height_restriction_ft'] + 1)

ratio_cols = ['rent_to_income_ratio', 'rent_to_home_value_ratio', 'rent_to_land_value',
              'tower_density_per_capita', 'safmr_premium', 'capacity_per_tenant',
              'height_utilization']
print(f"Ratio features created: {len(ratio_cols)}")
for col in ratio_cols:
    print(f"  {col}: median={dfe[col].median():.4f}")

In [ ]:
# ---- 5.4 Composite Index Features ----

# Market Attractiveness Index (combines demographic signals)
dfe['market_attractiveness'] = (
    StandardScaler().fit_transform(dfe[['census_median_income']]) * 0.3 +
    StandardScaler().fit_transform(dfe[['census_median_home_value']]) * 0.3 +
    StandardScaler().fit_transform(dfe[['census_population_density']]) * 0.2 +
    StandardScaler().fit_transform(dfe[['underlying_land_value_psf']]) * 0.2
).ravel()

# Site Strategic Value Index (network importance)
dfe['strategic_value'] = (
    dfe['coverage_critical'] * 3 +
    dfe['tenant_count'] * 1.5 +
    (1 - dfe['network_substitutability_score']) * 2 +
    dfe['high_traffic'] * 1 +
    dfe['has_fiber_connection'] * 1
)

# Competition Pressure Index
dfe['competition_pressure'] = (
    StandardScaler().fit_transform(dfe[['fcc_tower_count_5km']]) * 0.4 +
    StandardScaler().fit_transform(dfe[['landlord_concentration']]) * 0.3 -
    StandardScaler().fit_transform(dfe[['competitor_tower_distance_km']]) * 0.3
).ravel()

# Regulatory Burden Index
dfe['regulatory_burden'] = (
    StandardScaler().fit_transform(dfe[['wrluri_index']]) * 0.4 +
    StandardScaler().fit_transform(dfe[['permit_approval_difficulty']].astype(float)) * 0.4 +
    dfe['colocation_required'] * 0.5 -
    dfe['has_zoning_data'] * 0.2
)

# Escalation Drift Index (how much rent has drifted from origination)
dfe['escalation_drift'] = np.log1p(dfe['cumulative_escalation_factor'] - 1) * dfe['lease_age_years']

composite_cols = ['market_attractiveness', 'strategic_value', 'competition_pressure',
                  'regulatory_burden', 'escalation_drift']
print(f"Composite indices created: {len(composite_cols)}")
for col in composite_cols:
    print(f"  {col}: mean={dfe[col].mean():.3f}, std={dfe[col].std():.3f}")

In [ ]:
# ---- 5.5 Categorical Encoding (target-free) ----

# Label encode categorical features
le_cols = {}
for col in ['property_type', 'structure_type', 'zoning_district_type', 'escalation_type']:
    le = LabelEncoder()
    dfe[f'{col}_encoded'] = le.fit_transform(dfe[col])
    le_cols[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"  {col} -> {col}_encoded: {le_cols[col]}")

# One-hot encode property_type (most important categorical)
prop_dummies = pd.get_dummies(dfe['property_type'], prefix='pt', dtype=int)
dfe = pd.concat([dfe, prop_dummies], axis=1)
print(f"\nOne-hot property type columns: {list(prop_dummies.columns)}")

# State frequency encoding (number of sites per state / total)
state_freq = dfe['state'].value_counts(normalize=True)
dfe['state_frequency'] = dfe['state'].map(state_freq)
print(f"\nState frequency encoding: range [{dfe['state_frequency'].min():.4f}, {dfe['state_frequency'].max():.4f}]")

# Metro region frequency encoding
metro_freq = dfe['metro_region'].value_counts(normalize=True)
dfe['metro_frequency'] = dfe['metro_region'].map(metro_freq)

print(f"\nTotal columns after encoding: {len(dfe.columns)}")

In [ ]:
# ---- 5.6 Feature Importance Screening ----
# Use mutual information regression to rank features by predictive value

# Exclude ID, target-adjacent, and ground-truth columns
exclude_for_mi = ['site_id', 'lat', 'lon', 'state', 'metro_region',
                  'property_type', 'structure_type', 'zoning_district_type',
                  'escalation_type', 'true_fmv', 'true_deviation_pct',
                  'true_class', 'current_monthly_rent', 'de_escalated_base_rent']

mi_features = [c for c in dfe.select_dtypes(include=[np.number]).columns
               if c not in exclude_for_mi]

X_mi = dfe[mi_features].fillna(0).values
y_mi = dfe['current_monthly_rent'].values

mi_scores = mutual_info_regression(X_mi, y_mi, random_state=42, n_neighbors=5)
mi_df = pd.DataFrame({'feature': mi_features, 'mi_score': mi_scores})
mi_df = mi_df.sort_values('mi_score', ascending=False).reset_index(drop=True)

# Plot top 30
fig, ax = plt.subplots(figsize=(10, 12))
top_30 = mi_df.head(30)
ax.barh(top_30['feature'][::-1], top_30['mi_score'][::-1], color='steelblue', alpha=0.7)
ax.set_xlabel('Mutual Information Score')
ax.set_title('Top 30 Features by Mutual Information with Rent')
plt.tight_layout()
plt.show()

print("Top 20 features by mutual information:")
for _, row in mi_df.head(20).iterrows():
    print(f"  {row['feature']}: MI={row['mi_score']:.4f}")

In [ ]:
# ---- 5.7 VIF-Based Multicollinearity Check ----
# Check top 25 features by MI for multicollinearity

top_vif_features = mi_df.head(25)['feature'].tolist()
# Remove any features with zero variance
X_vif = dfe[top_vif_features].fillna(0)
X_vif = X_vif.loc[:, X_vif.std() > 0]
X_vif_scaled = StandardScaler().fit_transform(X_vif)

vif_data = []
for i, col in enumerate(X_vif.columns):
    try:
        vif = variance_inflation_factor(X_vif_scaled, i)
        vif_data.append({'feature': col, 'vif': vif})
    except:
        vif_data.append({'feature': col, 'vif': np.nan})

vif_df = pd.DataFrame(vif_data).sort_values('vif', ascending=False)
print("VIF Analysis (>10 = high multicollinearity, >5 = moderate):")
print("-" * 50)
for _, row in vif_df.iterrows():
    flag = " ***HIGH***" if row['vif'] > 10 else " *moderate*" if row['vif'] > 5 else ""
    print(f"  {row['feature']:40s} VIF={row['vif']:8.2f}{flag}")

high_vif = vif_df[vif_df['vif'] > 10]['feature'].tolist()
print(f"\nFeatures with VIF > 10: {len(high_vif)}")
if high_vif:
    print(f"  Consider dropping or combining: {high_vif}")

In [ ]:
# ---- 5.8 Final Feature Selection ----
# Select features for modeling based on MI ranking and VIF analysis

# Tier 1: Top MI features that aren't high-VIF (keep at least the originals)
# We keep all engineered features and let the model (XGBoost) handle importance

# Define the final modeling feature set
FINAL_FEATURES = {
    'numeric_original': [
        'tower_height_ft', 'building_height_ft', 'lease_area_sqft', 'parcel_acreage',
        'construction_age', 'site_access_ordinal',
        'census_population_density', 'census_population_3km',
        'census_median_income', 'census_median_home_value', 'census_median_rent',
        'census_vacancy_rate', 'urban_rural_class', 'underlying_land_value_psf',
        'hud_safmr', 'property_tax_rate', 'topography_class',
        'terrain_elevation_variance_m', 'poi_density_1km', 'building_density_500m',
        'distance_to_highway_km', 'ground_elevation_ft', 'unemployment_rate_local',
        'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
        'competitor_tower_distance_km', 'tower_per_capita', 'scarcity_index',
        'landlord_concentration', 'composite_hazard_score',
        'tenant_count', 'primary_technology', 'coverage_critical',
        'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
        'has_fiber_connection', 'network_substitutability_score',
        'search_ring_alternative_count', 'inter_site_distance_km',
        'antenna_count', 'structure_capacity_remaining', 'site_priority_tier',
        'permit_approval_difficulty', 'wrluri_index', 'colocation_required',
        'height_restriction_ft', 'has_zoning_data',
    ],
    'binary': [
        'has_building', 'is_ground_tower', 'has_parcel',
    ],
    'log_transforms': log_created,
    'interactions': interaction_cols,
    'ratios': ratio_cols,
    'composites': composite_cols,
    'encoded': [f'{c}_encoded' for c in ['property_type', 'structure_type',
                'zoning_district_type', 'escalation_type']] + list(prop_dummies.columns),
    'frequency': ['state_frequency', 'metro_frequency'],
    'lease_contract': [
        'lease_age_years', 'remaining_term_years', 'escalation_rate',
        'cumulative_escalation_factor', 'amendment_count',
    ],
}

all_features = []
for group, cols in FINAL_FEATURES.items():
    valid = [c for c in cols if c in dfe.columns]
    all_features.extend(valid)
    print(f"  {group}: {len(valid)} features")

all_features = list(dict.fromkeys(all_features))  # deduplicate, preserve order
print(f"\nTotal modeling features: {len(all_features)}")

# Verify all exist
missing_feats = [f for f in all_features if f not in dfe.columns]
if missing_feats:
    print(f"  WARNING: Missing features: {missing_feats}")
    all_features = [f for f in all_features if f in dfe.columns]

In [ ]:
# Build final dataset with selected features + identity + target + ground truth
id_cols = ['site_id', 'lat', 'lon', 'state', 'metro_region']
target_cols = ['current_monthly_rent', 'de_escalated_base_rent']
gt_cols = ['true_fmv', 'true_deviation_pct', 'true_class']
cat_cols_keep = ['property_type', 'structure_type', 'escalation_type',
                 'zoning_district_type']

final_cols = id_cols + cat_cols_keep + all_features + target_cols + gt_cols
final_cols = list(dict.fromkeys(final_cols))  # deduplicate

df_final = dfe[final_cols].copy()
print(f"Final engineered dataset: {df_final.shape}")
print(f"  Feature columns: {len(all_features)}")
print(f"  Total columns: {len(final_cols)}")

# Save
eng_path = os.path.join(PROC_DIR, "sites_engineered.parquet")
df_final.to_parquet(eng_path, index=False)
print(f"\nSaved to {eng_path}")
print(f"  File size: {os.path.getsize(eng_path) / 1024:.0f} KB")

# Also save the feature list for downstream notebooks
import json as _json
feat_meta = {
    'feature_groups': {k: v for k, v in FINAL_FEATURES.items()},
    'total_features': len(all_features),
    'all_features': all_features,
}
feat_meta_path = os.path.join(PROC_DIR, "feature_metadata.json")
with open(feat_meta_path, 'w') as f:
    _json.dump(feat_meta, f, indent=2)
print(f"  Feature metadata: {feat_meta_path}")

---
## Summary

### What was done

| Phase | Output | Key metrics |
|-------|--------|-------------|
| **Data Assembly** | 5 000 synthetic sites + optional enrichment merge | 72 raw columns |
| **EDA** | Target analysis, correlations, geographic patterns, class separability | Top predictors identified |
| **Data Cleaning** | Missing value imputation, sentinel removal, outlier winsorization, type validation | Clean dataset saved |
| **Hypothesis Testing** | 9 formal hypotheses with BH-FDR and Bonferroni correction | See summary table above |
| **Feature Engineering** | Log transforms, interactions, ratios, composites, encodings | 100+ model-ready features |

### Output Files

| File | Description |
|------|-------------|
| `data/processed/sites_clean.parquet` | Cleaned dataset (pre-engineering) |
| `data/processed/sites_engineered.parquet` | Feature-engineered, model-ready dataset |
| `data/processed/feature_metadata.json` | Feature groups and full feature list |

### Next Steps
- **Notebook 02**: Clustering & pseudo-label generation using the engineered features
- **Notebook 03**: Supervised FMV modeling (XGBoost + QRF + Ridge meta-learner)
- **Notebook 04**: Validation & SHAP interpretation